## Data Cleaing

1. convert all production dimentions to metric (grams, meters, CM)
2. Ignore unwanted columns that have only one value, e.g., Online_Order_Type and Online_Order_Type_Desc have value “ONLINE”
3. break down data and time columns to add in week number, month number, year, ...
4. bring in holidays into the dataset (Black Friday, etc.)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('ds_challenge_data_fixed.csv')

In [ ]:
#output cleared
df.sort_values(by=['Hash_Trans_ID'], inplace=False)

In [4]:
df.columns

Index(['Hash_Trans_ID', 'SKU_ID', 'Transaction_Date_ID', 'Transaction_Hour_ID',
       'Online_Order_Type', 'Online_Order_Type_Desc', 'Transaction_Type',
       'Transaction_Sales_Type', 'POS_Sales', 'POS_UnitsSold', 'Department_ID',
       'Class_ID', 'SubClass_ID', 'SubClass_Desc', 'Product_Weight',
       'Product_Weight_Dimension', 'Product_Height', 'Product_Length',
       'Product_Width', 'Product_Unit_Dimension', 'Product_Volumn',
       'Product_Volumn_Unit', 'PRO'],
      dtype='object')

### Cleanup the transaction id

In [ ]:
#output cleared
df[df['Hash_Trans_ID']=='0006d7dd4c3fd47287aca3bbeb1976cb4edc4a7d']

In [6]:
#no need for this anymore
#df['Cleaned_Transaction_ID'] = df[['Transaction_ID','Transaction_Date_ID','Transaction_Hour_ID']].apply(
#    lambda x: '-'.join(x.astype(str)),
#    axis=1
#)

In [7]:
test1 = df.groupby('Hash_Trans_ID').agg({'PRO': np.min}).reset_index().rename({'PRO': 'MIN_PRO'}, axis=1)
test2 = df.groupby('Hash_Trans_ID').agg({'PRO': np.max}).reset_index().rename({'PRO': 'MAX_PRO'}, axis=1)

In [8]:
test = pd.merge(test1, test2, how='inner', on = 'Hash_Trans_ID')

In [9]:
test[test['MIN_PRO']!=test['MAX_PRO']]

,Hash_Trans_ID,MIN_PRO,MAX_PRO


Now - with new HashID everything seems to be fine


Old - At least seems that PROs are unique for these IDs, but this might create an issue where both PROs are the same but it is actually same transaction happened in a short amount of time from each other

### Convert everything to metric

In [10]:
df.columns

Index(['Hash_Trans_ID', 'SKU_ID', 'Transaction_Date_ID', 'Transaction_Hour_ID',
       'Online_Order_Type', 'Online_Order_Type_Desc', 'Transaction_Type',
       'Transaction_Sales_Type', 'POS_Sales', 'POS_UnitsSold', 'Department_ID',
       'Class_ID', 'SubClass_ID', 'SubClass_Desc', 'Product_Weight',
       'Product_Weight_Dimension', 'Product_Height', 'Product_Length',
       'Product_Width', 'Product_Unit_Dimension', 'Product_Volumn',
       'Product_Volumn_Unit', 'PRO'],
      dtype='object')

#### Prod Weight

In [11]:
df['Product_Weight_Dimension'].value_counts()

LB    316813
KG      4989
G         34
OZ        11
Name: Product_Weight_Dimension, dtype: int64

In [12]:
df['Product_Weight_Grams'] = df[['Product_Weight','Product_Weight_Dimension']].apply(
    lambda x: x[0]*1000 if x[1]=='KG' else (x[0]*453.592 if x[1]=='LB' else (x[0]*28.35 if x[1]=='OZ' else x[0])),
    axis=1
)

In [ ]:
#test to see if it worked - cleared
df[ (df['Product_Weight_Dimension']=='KG') & (df['Product_Weight']!=0)]

#### Prod Volume

In [14]:
df['Product_Volumn_Unit'].value_counts()

FT3    307703
0       14144
Name: Product_Volumn_Unit, dtype: int64

In [15]:
df['Product_Volumn_Metric'] = df[['Product_Volumn','Product_Volumn_Unit']].apply(
    lambda x: x[0]*10.764 if x[1]=='FT3' else  x[0],
    axis=1
)

#### Prod Unit Dims

In [16]:
df['Product_Unit_Dimension'].value_counts()

IN    316841
0       5000
FT         4
CM         2
Name: Product_Unit_Dimension, dtype: int64

In [17]:
df['Product_Height_CM'] = df[['Product_Height','Product_Unit_Dimension']].apply(
    lambda x: x[0]*30.48 if x[1]=='FT' else (x[0]*2.54 if x[1]=='IN' else x[0]),
    axis=1
)

In [18]:
df['Product_Length_CM'] = df[['Product_Length','Product_Unit_Dimension']].apply(
    lambda x: x[0]*30.48 if x[1]=='FT' else (x[0]*2.54 if x[1]=='IN' else x[0]),
    axis=1
)

In [19]:
df['Product_Width_CM'] = df[['Product_Width','Product_Unit_Dimension']].apply(
    lambda x: x[0]*30.48 if x[1]=='FT' else (x[0]*2.54 if x[1]=='IN' else x[0]),
    axis=1
)

## Keep only the columns that are required so far

In [20]:
df.columns

Index(['Hash_Trans_ID', 'SKU_ID', 'Transaction_Date_ID', 'Transaction_Hour_ID',
       'Online_Order_Type', 'Online_Order_Type_Desc', 'Transaction_Type',
       'Transaction_Sales_Type', 'POS_Sales', 'POS_UnitsSold', 'Department_ID',
       'Class_ID', 'SubClass_ID', 'SubClass_Desc', 'Product_Weight',
       'Product_Weight_Dimension', 'Product_Height', 'Product_Length',
       'Product_Width', 'Product_Unit_Dimension', 'Product_Volumn',
       'Product_Volumn_Unit', 'PRO', 'Product_Weight_Grams',
       'Product_Volumn_Metric', 'Product_Height_CM', 'Product_Length_CM',
       'Product_Width_CM'],
      dtype='object')

In [21]:
df['Online_Order_Type'].value_counts()

IN-STORE    321847
Name: Online_Order_Type, dtype: int64

In [22]:
df['Online_Order_Type_Desc'].value_counts()

IN-STORE    321847
Name: Online_Order_Type_Desc, dtype: int64

In [23]:
df = df[['Hash_Trans_ID','SKU_ID', 'Transaction_Date_ID', 'Transaction_Hour_ID','Transaction_Type',\
        'Transaction_Sales_Type', 'POS_Sales', 'POS_UnitsSold', 'Department_ID',\
        'Class_ID', 'SubClass_ID', 'SubClass_Desc', 'Product_Weight_Grams', \
        'Product_Height_CM', 'Product_Length_CM','Product_Width_CM', \
         'Product_Volumn_Metric','PRO']]

In [24]:
df.describe()

,SKU_ID,Transaction_Date_ID,Transaction_Hour_ID,POS_Sales,POS_UnitsSold,Department_ID,Class_ID,SubClass_ID,Product_Weight_Grams,Product_Height_CM,Product_Length_CM,Product_Width_CM,Product_Volumn_Metric,PRO
count,321847.000000,3.218470e+05,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,3.218470e+05,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000
mean,170075.000251,2.014561e+07,133308.478174,286.996263,3.508021,26.010691,2144.219135,11090.473119,3.448497e+03,42.091758,38.152380,68.614179,8.706062,0.521614
std,46.324207,6.927640e+03,34878.750918,1797.485359,26.829688,7.501665,87.283612,762.115505,1.075314e+04,367.240360,72.545995,1784.488903,147.264807,0.499533
min,170017.016500,2.013051e+07,52032.000000,-84000.000000,-2000.000000,11.000000,2001.000000,10001.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,170024.639800,2.014082e+07,104048.000000,31.740000,1.000000,24.000000,2086.000000,10554.000000,0.000000e+00,2.540000,2.540000,5.080000,0.000000,0.000000
50%,170072.638800,2.015032e+07,131635.000000,81.720000,1.000000,25.000000,2123.000000,10866.000000,4.535920e+02,10.160000,10.160000,10.160000,0.000000,1.000000
75%,170121.466500,2.015072e+07,160242.500000,215.640000,2.000000,27.000000,2202.000000,11463.000000,1.814368e+03,20.320000,30.480000,20.320000,0.000000,1.000000
max,170169.996300,2.015123e+07,234944.000000,545225.340000,3456.000000,78.000000,2417.000000,13043.000000,1.360776e+06,21752.560000,3078.480000,201391.520000,39256.308000,1.000000


In [25]:
df.dtypes

Hash_Trans_ID              object
SKU_ID                    float64
Transaction_Date_ID         int64
Transaction_Hour_ID         int64
Transaction_Type           object
Transaction_Sales_Type     object
POS_Sales                 float64
POS_UnitsSold             float64
Department_ID               int64
Class_ID                    int64
SubClass_ID                 int64
SubClass_Desc              object
Product_Weight_Grams      float64
Product_Height_CM         float64
Product_Length_CM         float64
Product_Width_CM          float64
Product_Volumn_Metric     float64
PRO                         int64
dtype: object

## Cleanup Date and Time Columns

In [26]:
df['Transaction_Year'] = df.Transaction_Date_ID.astype(str).str.slice(0,4).astype(int)

In [27]:
df['Transaction_Month'] = df.Transaction_Date_ID.astype(str).str.slice(4,6).astype(int)

In [28]:
df['Transaction_Day'] = df.Transaction_Date_ID.astype(str).str.slice(6,8).astype(int)

#### break down the time column

In [29]:
df['Transaction_Time'] = df['Transaction_Hour_ID'].astype(str).apply(
    lambda x: '{0:0>6}'.format(x)
)

In [30]:
df['Transaction_Hour'] = df.Transaction_Time.astype(str).str.slice(0,2).astype(int)

In [31]:
df['Transaction_Min'] = df.Transaction_Time.astype(str).str.slice(2,4).astype(int)

In [32]:
df['Transaction_Sec'] = df.Transaction_Time.astype(str).str.slice(4,6).astype(int)

In [33]:
df.describe()

,SKU_ID,Transaction_Date_ID,Transaction_Hour_ID,POS_Sales,POS_UnitsSold,Department_ID,Class_ID,SubClass_ID,Product_Weight_Grams,Product_Height_CM,Product_Length_CM,Product_Width_CM,Product_Volumn_Metric,PRO,Transaction_Year,Transaction_Month,Transaction_Day,Transaction_Hour,Transaction_Min,Transaction_Sec
count,321847.000000,3.218470e+05,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,3.218470e+05,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000,321847.000000
mean,170075.000251,2.014561e+07,133308.478174,286.996263,3.508021,26.010691,2144.219135,11090.473119,3.448497e+03,42.091758,38.152380,68.614179,8.706062,0.521614,2014.489667,6.996952,15.853607,13.029197,29.870392,29.467961
std,46.324207,6.927640e+03,34878.750918,1797.485359,26.829688,7.501665,87.283612,762.115505,1.075314e+04,367.240360,72.545995,1784.488903,147.264807,0.499533,0.700042,3.316466,8.777957,3.498879,17.307577,17.319502
min,170017.016500,2.013051e+07,52032.000000,-84000.000000,-2000.000000,11.000000,2001.000000,10001.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,2013.000000,1.000000,1.000000,5.000000,0.000000,0.000000
25%,170024.639800,2.014082e+07,104048.000000,31.740000,1.000000,24.000000,2086.000000,10554.000000,0.000000e+00,2.540000,2.540000,5.080000,0.000000,0.000000,2014.000000,4.000000,8.000000,10.000000,15.000000,14.000000
50%,170072.638800,2.015032e+07,131635.000000,81.720000,1.000000,25.000000,2123.000000,10866.000000,4.535920e+02,10.160000,10.160000,10.160000,0.000000,1.000000,2015.000000,7.000000,16.000000,13.000000,30.000000,29.000000
75%,170121.466500,2.015072e+07,160242.500000,215.640000,2.000000,27.000000,2202.000000,11463.000000,1.814368e+03,20.320000,30.480000,20.320000,0.000000,1.000000,2015.000000,10.000000,23.000000,16.000000,45.000000,44.000000
max,170169.996300,2.015123e+07,234944.000000,545225.340000,3456.000000,78.000000,2417.000000,13043.000000,1.360776e+06,21752.560000,3078.480000,201391.520000,39256.308000,1.000000,2015.000000,12.000000,31.000000,23.000000,59.000000,59.000000


In [34]:
df['Transaction_Date']=df[['Transaction_Year','Transaction_Month','Transaction_Day','Transaction_Hour','Transaction_Min','Transaction_Sec']].apply(
    lambda x: datetime(x[0],x[1],x[2],x[3],x[4],x[5]),
    axis=1
)

In [35]:
df['Transaction_Date'] = pd.to_datetime(df['Transaction_Date'])

In [36]:
df['Transaction_Day_of_Week'] = df['Transaction_Date'].dt.dayofweek

In [37]:
df['Transaction_Week'] = df['Transaction_Date'].dt.weekofyear

In [38]:
df['Transaction_Day_of_Year'] = df['Transaction_Date'].dt.dayofyear

### Add in Holidays to the dataset

In [39]:
from datetime import date 
import holidays

In [40]:
hldy = holidays.Canada()

In [41]:
df['holiday_nm'] = df['Transaction_Date'].apply(lambda x: hldy.get(x))

In [42]:
df.columns

Index(['Hash_Trans_ID', 'SKU_ID', 'Transaction_Date_ID', 'Transaction_Hour_ID',
       'Transaction_Type', 'Transaction_Sales_Type', 'POS_Sales',
       'POS_UnitsSold', 'Department_ID', 'Class_ID', 'SubClass_ID',
       'SubClass_Desc', 'Product_Weight_Grams', 'Product_Height_CM',
       'Product_Length_CM', 'Product_Width_CM', 'Product_Volumn_Metric', 'PRO',
       'Transaction_Year', 'Transaction_Month', 'Transaction_Day',
       'Transaction_Time', 'Transaction_Hour', 'Transaction_Min',
       'Transaction_Sec', 'Transaction_Date', 'Transaction_Day_of_Week',
       'Transaction_Week', 'Transaction_Day_of_Year', 'holiday_nm'],
      dtype='object')

In [43]:
df = df[['Hash_Trans_ID','SKU_ID', 'Transaction_Date', 'Transaction_Year', 'Transaction_Month', 'Transaction_Day',\
         'Transaction_Hour','Transaction_Min', 'Transaction_Sec','Transaction_Day_of_Week',\
       'Transaction_Week', 'Transaction_Day_of_Year','holiday_nm','Transaction_Type',\
        'Transaction_Sales_Type', 'POS_Sales', 'POS_UnitsSold', 'Department_ID',\
        'Class_ID', 'SubClass_ID', 'SubClass_Desc', 'Product_Weight_Grams', \
        'Product_Height_CM', 'Product_Length_CM','Product_Width_CM', \
         'Product_Volumn_Metric','PRO']]

In [44]:
df.to_csv('ds_challenge_data_cleaned.csv', index=False)